## 🔽 Download ArcFace Model Automatically

If the ArcFace model is not found locally, the following script will automatically download it from Hugging Face:


In [11]:
import os
import requests

ARC_URL = "https://huggingface.co/garavv/arcface-onnx/resolve/main/arc.onnx?download=true"
ARC_LOCAL = r"C:\Users\Hasnain\Desktop\DeepFaceSwap\model\arcface.onnx"

def download_arcface():
    if not os.path.exists(ARC_LOCAL):
        print("ArcFace model not found, downloading...")
        resp = requests.get(ARC_URL, stream=True)
        if resp.status_code == 200:
            with open(ARC_LOCAL, "wb") as f:
                for chunk in resp.iter_content(1024 * 1024):
                    f.write(chunk)
            print("Downloaded ArcFace ONNX model.")
        else:
            raise RuntimeError(f"Failed to download ArcFace model, status code {resp.status_code}")

# Usage in your main script
download_arcface()


ArcFace model not found, downloading...
Downloaded ArcFace ONNX model.


## 🖌️ Poisson Blending (Seamless Face Swap)

This function uses **OpenCV’s `cv2.seamlessClone`** to blend the swapped face into the target image smoothly, avoiding harsh edges or ghosting artifacts.


In [19]:
def poisson_blend(full, patch, box, blur_sigma=15, mode=cv2.NORMAL_CLONE):
    """
    Poisson (seamless) blend 'patch' into 'full' inside the rectangle 'box'.
    box = (x1, y1, x2, y2)
    """
    x1, y1, x2, y2 = box
    H, W = y2 - y1, x2 - x1

    # 1) Resize patch to box size
    src = cv2.resize(patch, (W, H))

    # 2) Build a 1-channel 8-bit mask same size as src (white = keep, black = ignore)
    mask = np.ones((H, W), dtype=np.uint8) * 255
    if blur_sigma > 0:
        mask = cv2.GaussianBlur(mask, (0, 0), blur_sigma)

    # 3) Place src over a black canvas the size of full image
    src_canvas = np.zeros_like(full)
    src_canvas[y1:y2, x1:x2] = src

    # 4) Place mask over a black canvas too (must be 1-channel 8-bit)
    mask_canvas = np.zeros(full.shape[:2], dtype=np.uint8)
    mask_canvas[y1:y2, x1:x2] = mask

    # 5) Choose a center for seamlessClone (must lie INSIDE the mask region)
    center = ((x1 + x2) // 2, (y1 + y2) // 2)

    # 6) Poisson blend
    blended = cv2.seamlessClone(src_canvas, full, mask_canvas, center, mode)
    return blended


In [20]:
final = poisson_blend(full, swapped, tgt_box, blur_sigma=15, mode=cv2.NORMAL_CLONE)
cv2.imwrite(OUTPUT_IMAGE, final)


True

## 🧪 Environment Test Script (ONNX + ONNXRuntime)

Before running the full face swap pipeline, it’s useful to test whether **ONNXRuntime** is correctly set up with CUDA or CPU.  
This script builds a tiny **dummy ONNX model**

In [21]:
import onnx
import onnxruntime as ort
import numpy as np

print("🔍 Checking ONNXRuntime providers...")
print("Available providers:", ort.get_available_providers())

# Pick CUDA if available, otherwise CPU
provider = "CUDAExecutionProvider" if "CUDAExecutionProvider" in ort.get_available_providers() else "CPUExecutionProvider"
print(f"✅ Using provider: {provider}")

# Build a dummy ONNX model with low IR/opset version
X = onnx.helper.make_tensor_value_info("input", onnx.TensorProto.FLOAT, [1, 3])
Y = onnx.helper.make_tensor_value_info("output", onnx.TensorProto.FLOAT, [1, 3])
node = onnx.helper.make_node("Identity", inputs=["input"], outputs=["output"])
graph = onnx.helper.make_graph([node], "IdentityGraph", [X], [Y])
model = onnx.helper.make_model(
    graph,
    producer_name="test",
    ir_version=7,  # <= 9 supported by your ORT
    opset_imports=[onnx.helper.make_opsetid("", 9)]
)

# Save model
onnx.save(model, "dummy.onnx")

# Load into session
session = ort.InferenceSession("dummy.onnx", providers=[provider])

# Test inference
inp = np.array([[1.0, 2.0, 3.0]], dtype=np.float32)
out = session.run(None, {"input": inp})[0]

print("🔧 Input :", inp)
print("📦 Output:", out)
print("🎉 Test completed successfully!")


🔍 Checking ONNXRuntime providers...
Available providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
✅ Using provider: CUDAExecutionProvider
🔧 Input : [[1. 2. 3.]]
📦 Output: [[1. 2. 3.]]
🎉 Test completed successfully!


## 🎥 Real-Time Face Swap (Webcam Demo)

This script runs **SimSwap ONNX** + **ArcFace ONNX** with OpenCV and MediaPipe to perform **real-time face swapping** from your webcam.  

### 📌 Features
- Uses **ArcFace** to extract identity embedding from a source image.  
- Uses **SimSwap ONNX** (`simswap_256.onnx`) to generate the swapped face.  
- Supports two blending methods:
  - **Poisson blending** (smooth, natural look using `cv2.seamlessClone`)  
  - **Feather blending** (lighter but less realistic).  
- Works in **real-time** on CUDA (`CUDAExecutionProvider`) or CPU fallback.  

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
import mediapipe as mp
import time

# ========= EDIT THESE PATHS =========
# replace username with your real username
SIMSWAP_MODEL = r"C:\Users\username\Desktop\DeepFaceSwap\model\simswap_256.onnx"
ARCFACE_MODEL = r"C:\Users\username\Desktop\DeepFaceSwap\model\arcface.onnx"
SOURCE_IMAGE  = r"C:\Users\username\Desktop\DeepFaceSwap\source.jpg"  # identity
# ====================================

BLEND_MODE = "poisson"   # "feather" or "poisson"

# ---------- Face utils ----------
def detect_and_crop(img):
    with mp.solutions.face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.6) as det:
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        res = det.process(rgb)
        if not res.detections:
            return None, None
        b = res.detections[0].location_data.relative_bounding_box
        H, W = img.shape[:2]
        x, y, w, h = int(b.xmin*W), int(b.ymin*H), int(b.width*W), int(b.height*H)
        s = int(max(w, h) * 1.6)
        cx, cy = x + w//2, y + h//2
        x1, y1 = max(0, cx - s//2), max(0, cy - s//2)
        x2, y2 = min(W, cx + s//2), min(H, cy + s//2)
        return img[y1:y2, x1:x2].copy(), (x1, y1, x2, y2)

def preprocess_face_256(bgr):
    im = cv2.resize(bgr, (256, 256)).astype(np.float32) / 255.0
    return np.transpose(im, (2, 0, 1))[None, ...]

def preprocess_arcface_NHWC(bgr):
    im = cv2.resize(bgr, (112, 112))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB).astype(np.float32)
    im = (im - 127.5) / 128.0
    return im[None, ...]

def postprocess_face(out_tensor):
    out = out_tensor[0] if out_tensor.ndim == 4 else out_tensor
    out = np.transpose(out, (1, 2, 0))
    out = (np.clip(out, 0, 1) if out.max() <= 1.5 else np.clip(out/255.0, 0, 1)) * 255
    return out.astype(np.uint8)

def feather_blend(full, patch, box, feather=20):
    x1, y1, x2, y2 = box
    H, W = y2 - y1, x2 - x1
    patch = cv2.resize(patch, (W, H))
    mask = np.ones((H, W), dtype=np.float32)
    mask = cv2.GaussianBlur(mask, (0, 0), feather)
    mask /= (mask.max() + 1e-6)
    mask = mask[..., None]
    roi = full[y1:y2, x1:x2].astype(np.float32)
    blended = (roi * (1.0 - mask) + patch.astype(np.float32) * mask).astype(np.uint8)
    out = full.copy()
    out[y1:y2, x1:x2] = blended
    return out

def poisson_blend(full, patch, box, blur_sigma=15, mode=cv2.NORMAL_CLONE):
    x1, y1, x2, y2 = box
    H, W = y2 - y1, x2 - x1
    src = cv2.resize(patch, (W, H))
    mask = np.ones((H, W), dtype=np.uint8) * 255
    if blur_sigma > 0:
        mask = cv2.GaussianBlur(mask, (0, 0), blur_sigma)
    src_canvas = np.zeros_like(full)
    src_canvas[y1:y2, x1:x2] = src
    mask_canvas = np.zeros(full.shape[:2], dtype=np.uint8)
    mask_canvas[y1:y2, x1:x2] = mask
    center = ((x1 + x2) // 2, (y1 + y2) // 2)
    blended = cv2.seamlessClone(src_canvas, full, mask_canvas, center, mode)
    return blended

# ---------- Setup models ----------
providers = ["CUDAExecutionProvider"] if "CUDAExecutionProvider" in ort.get_available_providers() else ["CPUExecutionProvider"]
print("Using providers:", providers)

ss = ort.InferenceSession(SIMSWAP_MODEL, providers=providers)
ss_face_in, ss_id_in, ss_out_name = "input", "onnx::Gemm_1", "output"
af = ort.InferenceSession(ARCFACE_MODEL, providers=providers)
af_in_name, af_out_name = "input_1", "embedding"

# ---------- Precompute identity embedding ----------
src_img = cv2.imread(SOURCE_IMAGE)
src_face, _ = detect_and_crop(src_img)
src_in = preprocess_arcface_NHWC(src_face)
emb = af.run([af_out_name], {af_in_name: src_in})[0]
emb = emb / (np.linalg.norm(emb, axis=1, keepdims=True) + 1e-6)

# ---------- Webcam loop ----------
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("❌ Could not open webcam")

print("🎥 Press 'q' to quit.")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        tgt_face, tgt_box = detect_and_crop(frame)
        if tgt_face is not None:
            tgt_in = preprocess_face_256(tgt_face)
            result = ss.run([ss_out_name], {ss_face_in: tgt_in, ss_id_in: emb})[0]
            swapped = postprocess_face(result)

            if BLEND_MODE == "poisson":
                frame = poisson_blend(frame, swapped, tgt_box)
            else:
                frame = feather_blend(frame, swapped, tgt_box)
    except Exception as e:
        # skip frame if detection fails
        print("⚠️", e)

    cv2.imshow("SimSwap Live", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
